In [2]:
!pip install minio
!pip install boto3
!pip install pyspark requests
!pip install pyspark kafka-python
!pip install confluent_kafka
!pip install newsdataapi

     |████████████████████████████████| 93 kB 716 kB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 14.8 MB/s eta 0:00:01    |███████▏                        | 2.8 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 626 kB/s  eta 0:00:01
     |████████████████████████████████| 198 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 246 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 3.7 MB/s eta 0:00:01


In [ ]:
import json
import time
import requests
import re
from datetime import datetime
from io import BytesIO
from kafka import KafkaProducer, KafkaConsumer
from kafka.admin import KafkaAdminClient, NewTopic
from minio import Minio
from minio.error import S3Error
import threading

# MinIO and Kafka configuration
minio_url = "minio:9000"
access_key = "access_key"
secret_key = "secret_key"
kafka_bootstrap_servers = "broker:29092"
kafka_topic = "news"
news_api_key = "api_key"
# news_api_key = "api_key"

# Initialize Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=kafka_bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Initialize Kafka Admin Client and create topic if it doesn't exist
admin_client = KafkaAdminClient(bootstrap_servers=kafka_bootstrap_servers)
topic = NewTopic(name=kafka_topic, num_partitions=1, replication_factor=1)
try:
    admin_client.create_topics(new_topics=[topic], validate_only=False)
    print(f"Topic '{kafka_topic}' created successfully")
except Exception as e:
    print(f"Topic '{kafka_topic}' may already exist or an error occurred: {e}")

# Initialize MinIO Client
minio_client = Minio(
    minio_url,
    access_key=access_key,
    secret_key=secret_key,
    secure=False
)

def fetch_news(api_key, producer, kafka_topic, news_topic):
    url = f"https://newsapi.org/v2/everything?q={news_topic}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        if articles:
            for article in articles:
                producer.send(kafka_topic, value=article)
            producer.flush()
            print(f"Sent {len(articles)} articles to Kafka topic '{kafka_topic}'")
        else:
            print(f"No articles found for topic: {news_topic}")
    else:
        print(f"Error fetching news: {response.status_code}")

def upload_to_minio(article, news_topic):
    bucket_name = re.sub(r'[^\w]', '_', news_topic)  

    # Parse the published date
    published_date = datetime.fromisoformat(article['publishedAt'][:-1])
    folder_name = published_date.strftime("%Y%m%d") 

    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created.")
    except S3Error as e:
        print(f"Error creating bucket: {e}")
    
    title = re.sub(r'[^\w\s]', '_', article['title']).replace(' ', '_')
    object_name = f"{folder_name}/{title}-{published_date.strftime('%H%M%S')}.json"  

    article_json = json.dumps(article)
    article_bytes = article_json.encode('utf-8')
    article_stream = BytesIO(article_bytes)

    minio_client.put_object(
        bucket_name,
        object_name,
        article_stream,
        length=len(article_bytes),
        content_type='application/json'
    )
    
    print(f"Uploaded article to {bucket_name}/{object_name}")

def produce_news():
    while True:
        fetch_news(news_api_key, producer, kafka_topic, "paris")
        time.sleep(300)

def consume_news():
    consumer = KafkaConsumer(
        kafka_topic,
        bootstrap_servers=kafka_bootstrap_servers,
        value_deserializer=lambda x: json.loads(x.decode('utf-8')),
        auto_offset_reset='earliest',
        enable_auto_commit=True,
        group_id='olympics'
    )
    
    print(f"Listening for messages on topic '{kafka_topic}'...")
    try:
        for message in consumer:
            article = message.value
            upload_to_minio(article, "paris")
    except KeyboardInterrupt:
        print("Stopping consumer...")
    finally:
        consumer.close()

if __name__ == "__main__":
    # Start producer and consumer in separate threads
    producer_thread = threading.Thread(target=produce_news)
    consumer_thread = threading.Thread(target=consume_news)

    producer_thread.start()
    consumer_thread.start()

    producer_thread.join()
    consumer_thread.join()